In [1]:
%autoreload 2

In [2]:
from argparse import Namespace
from collections import defaultdict
import copy
import difflib
import gzip
import itertools
import os
import pickle
import sys
import typing

from IPython.display import display, Markdown, HTML
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tatsu
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import tqdm.notebook as tqdm
import sklearn
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.pipeline import Pipeline

sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../src'))
from src import fitness_energy_utils as utils
from src.fitness_energy_utils import NON_FEATURE_COLUMNS
from src.ast_counter_sampler import *
from src.ast_utils import cached_load_and_parse_games_from_file, load_games_from_file, _extract_game_id
from src import ast_printer

In [3]:
fitness_df = utils.load_fitness_data()
print(fitness_df.src_file.unique())
fitness_df.head()

['interactive-beta.pddl' 'ast-real-regrowth-samples.pddl']


,Index,src_file,game_name,domain_name,all_variables_defined,all_variables_used,all_preferences_used,setup_objects_used,no_adjacent_once,starts_and_ends_once,...,mean_depth_setup,mean_depth_constraints,mean_depth_terminal,mean_depth_scoring,node_count_setup,node_count_constraints,node_count_terminal,node_count_scoring,real,original_game_name
0,0,interactive-beta.pddl,6172feb1665491d1efbce164-0,medium-objects-room-v1,1.0,1.0,1.0,1.00,1.0,0.5,...,5.739130,8.538462,2.6,1.500000,23,78,10,4,True,6172feb1665491d1efbce164-0
1,1,interactive-beta.pddl,5f77754ba932fb2c4ba181d8-2,many-objects-room-v1,1.0,1.0,1.0,1.00,1.0,1.0,...,3.000000,8.710843,2.6,4.921053,10,83,10,38,True,5f77754ba932fb2c4ba181d8-2
2,2,interactive-beta.pddl,614b603d4da88384282967a7-3,many-objects-room-v1,1.0,1.0,1.0,0.00,1.0,0.0,...,0.000000,5.611111,0.0,2.500000,0,18,0,6,True,614b603d4da88384282967a7-3
3,3,interactive-beta.pddl,5bc79f652885710001a0e82a-5,few-objects-room-v1,1.0,1.0,1.0,0.00,1.0,1.0,...,0.000000,9.215686,0.0,1.500000,0,51,0,4,True,5bc79f652885710001a0e82a-5
4,4,interactive-beta.pddl,614dec67f6eb129c3a77defd-6,medium-objects-room-v1,1.0,1.0,1.0,0.25,1.0,1.0,...,5.264706,9.622449,0.0,4.531250,34,98,0,32,True,614dec67f6eb129c3a77defd-6


In [4]:
def create_filtered_df(df: pd.DataFrame, 
    synthetic_data_src_files: typing.Sequence[str] = ('interactive-beta.pddl', 'ast-real-regrowth-samples.pddl'),
    ) -> pd.DataFrame:
    syntethic_df = fitness_df[fitness_df.src_file.isin(synthetic_data_src_files)].reset_index(drop=True)
    syntethic_df.loc[syntethic_df.src_file == synthetic_data_src_files[0], 'real'] = 1
    return syntethic_df

filtered_fitness_df = create_filtered_df(fitness_df)

In [5]:
filtered_fitness_df.head()

,Index,src_file,game_name,domain_name,all_variables_defined,all_variables_used,all_preferences_used,setup_objects_used,no_adjacent_once,starts_and_ends_once,...,mean_depth_setup,mean_depth_constraints,mean_depth_terminal,mean_depth_scoring,node_count_setup,node_count_constraints,node_count_terminal,node_count_scoring,real,original_game_name
0,0,interactive-beta.pddl,6172feb1665491d1efbce164-0,medium-objects-room-v1,1.0,1.0,1.0,1.00,1.0,0.5,...,5.739130,8.538462,2.6,1.500000,23,78,10,4,1,6172feb1665491d1efbce164-0
1,1,interactive-beta.pddl,5f77754ba932fb2c4ba181d8-2,many-objects-room-v1,1.0,1.0,1.0,1.00,1.0,1.0,...,3.000000,8.710843,2.6,4.921053,10,83,10,38,1,5f77754ba932fb2c4ba181d8-2
2,2,interactive-beta.pddl,614b603d4da88384282967a7-3,many-objects-room-v1,1.0,1.0,1.0,0.00,1.0,0.0,...,0.000000,5.611111,0.0,2.500000,0,18,0,6,1,614b603d4da88384282967a7-3
3,3,interactive-beta.pddl,5bc79f652885710001a0e82a-5,few-objects-room-v1,1.0,1.0,1.0,0.00,1.0,1.0,...,0.000000,9.215686,0.0,1.500000,0,51,0,4,1,5bc79f652885710001a0e82a-5
4,4,interactive-beta.pddl,614dec67f6eb129c3a77defd-6,medium-objects-room-v1,1.0,1.0,1.0,0.25,1.0,1.0,...,5.264706,9.622449,0.0,4.531250,34,98,0,32,1,614dec67f6eb129c3a77defd-6


In [6]:
identical_regrown_games = defaultdict(list)

feature_columns = [c for c in filtered_fitness_df.columns if c not in NON_FEATURE_COLUMNS]

for original_game_name in tqdm.tqdm(filtered_fitness_df.original_game_name.unique()):
    original_game = filtered_fitness_df[filtered_fitness_df.game_name == original_game_name]
    regrown_games = filtered_fitness_df[np.logical_and(filtered_fitness_df.original_game_name == original_game_name, filtered_fitness_df.src_file == 'ast-real-regrowth-samples.pddl')]
    for _, regrown_game in regrown_games.iterrows():
        if np.all(original_game[feature_columns] == regrown_game[feature_columns]):
            identical_regrown_games[original_game_name].append(regrown_game.game_name)

100%|██████████| 98/98 [06:27<00:00,  3.96s/it]


In [7]:
print(len(identical_regrown_games), np.mean([len(v) for v in identical_regrown_games.values()]))

88 7.068181818181818


In [8]:
RANDOM_SEED = 33

args = Namespace(
    parse_counter=False, 
    counter_output_path=os.path.join('..', DEFAULT_COUNTER_OUTPUT_PATH),
    grammar_file=os.path.join('..', DEFAULT_GRAMMAR_FILE),
    random_seed=RANDOM_SEED,
)

grammar = open(args.grammar_file).read()
grammar_parser = tatsu.compile(grammar)

game_texts = {_extract_game_id(text): text for text in load_games_from_file('../dsl/interactive-beta.pddl') }
original_game_asts = {ast[1].game_name: ast for ast in cached_load_and_parse_games_from_file('../dsl/interactive-beta.pddl', grammar_parser, False, '..')}
regrown_game_texts = {_extract_game_id(text): text for text in load_games_from_file('../dsl/ast-real-regrowth-samples.pddl') }

In [19]:
html_diff = difflib.HtmlDiff(wrapcolumn=60)

def display_diff(original_game_id_or_index: typing.Union[str, int], regrown_game_id_or_index: typing.Union[str, int]):
    if isinstance(original_game_id_or_index, int):
        original_game_id = list(identical_regrown_games.keys())[original_game_id_or_index]
    else:
        original_game_id = original_game_id_or_index

    if isinstance(regrown_game_id_or_index, int):
        regrown_game_id = identical_regrown_games[original_game_id][regrown_game_id_or_index]
    else:
        regrown_game_id = regrown_game_id_or_index


    regrown_game_text = regrown_game_texts[regrown_game_id]
    diff = html_diff.make_file(original_game_text.splitlines(), regrown_game_text.splitlines())  #, context=True, numlines=0)
    display(HTML(diff))

display_diff(50, 0)


n,1,(define (game 56cf6e8d31a5bc0006e1cdf5-65) (:domain many-obj,n,1,(define (game 56cf6e8d31a5bc0006e1cdf5-65-46) (:domain many-
,>,ects-room-v1),,>,objects-room-v1)
,2,(:constraints,,2,(:constraints
,3,(and,,3,(and
,4,(preference ballOnBedAtEnd,,4,(preference ballOnBedAtEnd
t,5,(exists (?b - ball),t,5,(exists (?b - dodgeball)
,6,(at-end,,6,(at-end
,7,(on bed ?b),,7,(on bed ?b)
,8,),,8,)
,9,),,9,)
,10,),,10,)


In [ ]:
filtered_fitness_df[filtered_fitness_df.game_name.isin(('6172feb1665491d1efbce164-0', '6172feb1665491d1efbce164-0-14'))]